In [1]:
import os
import pandas as pd
import datetime
import pytz
import json
import numpy as np

from IPython.display import display, HTML

from invoice_v3 import get_month_thai, create_invoice, secure_filename

tz = pytz.timezone('Asia/Bangkok')

In [2]:
DIR = './Invoice/November-2563'
IMAGES_DIR = os.path.join( DIR, 'IMAGES')
PDF_DIR = os.path.join( DIR, 'PDF')

sheet_path = './summary/พฤศจิกายน 2563.xlsx'
start_invoice_number = 1023
fee = 0.15

start_date = datetime.date(2020, 11, 1)
end_date = datetime.date(2020, 11, 30)

In [3]:
float_list = [
    'โคตรแซ่บริมคลอง'
]

In [4]:
def read_job_excel(path):
    xls = pd.ExcelFile(path)
    df = xls.parse('jobs', header=0, skiprows=4, usecols='A:D,G' , converters={
        'ชื่อร้านค้า': lambda x: x.strip(),
        'ราคาสินค้า': lambda x: x if x else 0,
        'วันที่': lambda x: x.date()
    })
    return df

In [5]:
def read_shop_excel(path):
    xls = pd.ExcelFile(path)
    df = xls.parse('shops', header=0, skiprows=2, usecols='A,C:H,' , converters={
        'ชื่อร้านค้า': lambda x: x.strip(),
    })
    return df

In [6]:
def get_data(shop, invoice_number):
    shop_job_df = job_df.loc[job_df['ชื่อร้านค้า'] == shop]
    date_list = sorted(shop_job_df['วันที่'].unique())
    credit = 5000
    
    sum_ = 0
    fee_ = 0
    lines = []
    
    for d in date_list:
        order_df = shop_job_df.loc[shop_job_df['วันที่'] == d]
        
        total_day = order_df['ราคาสินค้า'].sum(axis=0)
        fee_day = int( np.sum( np.floor( order_df['ราคาสินค้า'].to_numpy() * fee + .5 ) ) )
        sum_ += total_day
        fee_ += fee_day
        
        credit -= fee_day
        
        desc = "%d %s %d ( %d รายการ )" % (d.day, get_month_thai(d.month), d.year + 543, len( order_df['ราคาสินค้า'] ))
        
        line = [
                desc,
                "%d" % (total_day),
                "%d" % (fee_day),
#                 "%d" % (credit),
               ]
        lines.append(line)
    
    data = {
        'title': secure_filename(shop),
        'today': today,
        'invoice_number': invoice_number,
        'shop_name': shop,
        'start_date': "%d %s %d" % (start_date.day, get_month_thai(start_date.month), start_date.year + 543),
        'end_date': "%d %s %d" % (end_date.day, get_month_thai(end_date.month), end_date.year + 543),
        
        'sum': sum_,
        'fee': fee_,
        'credit': credit,
        'detail': lines,
    }
    
#     print(fee_)
    
    return data

In [7]:
def get_data_float(shop, invoice_number):
    shop_job_df = job_df.loc[job_df['ชื่อร้านค้า'] == shop]
    date_list = sorted(shop_job_df['วันที่'].unique())
    
    sum_ = 0
    fee_ = 0
    lines = []
    
    for d in date_list:
        order_df = shop_job_df.loc[shop_job_df['วันที่'] == d]
        
        total_day = order_df['ราคาสินค้า'].sum(axis=0)
        fee_day = float(np.sum(order_df['ราคาสินค้า'].to_numpy() * fee ))
        sum_ += total_day
        fee_ += fee_day
        
        desc = "%d %s %d ( %d รายการ )" % (d.day, get_month_thai(d.month), d.year + 543, len( order_df['ราคาสินค้า'] ))
        
        line = [
                desc,
                "%d" % (total_day),
                "%.2f" % (fee_day),
               ]
        lines.append(line)
    
    data = {
        'title': secure_filename(shop),
        'today': today,
        'invoice_number': invoice_number,
        'shop_name': shop,
        'start_date': "%d %s %d" % (start_date.day, get_month_thai(start_date.month), start_date.year + 543),
        'end_date': "%d %s %d" % (end_date.day, get_month_thai(end_date.month), end_date.year + 543),
        
        'sum': sum_,
        'fee': "%.2f" % fee_,
        'detail': lines,
    }
    
#     print(fee_)
    
    return data

In [8]:
def process(shop, invoice_number):
    data = get_data(shop, invoice_number)
    file_path = os.path.join( PDF_DIR, "ใบเสร็จ %s.pdf" % secure_filename(shop) )
    create_invoice(file_path, data)

In [9]:
def process_float(shop, invoice_number):
    data = get_data_float(shop, invoice_number)
    file_path = os.path.join( PDF_DIR, "ใบเสร็จ %s.pdf" % secure_filename(shop) )
    create_invoice(file_path, data)

In [10]:
now = datetime.datetime.now(tz)
today = "%d %s %d" % (now.day, get_month_thai(now.month), now.year + 543)

In [15]:
job_df = read_job_excel(sheet_path)
job_df = job_df.dropna()

shop_df = read_shop_excel(sheet_path)

In [16]:
shop_df

,ชื่อร้านค้า,ยอดขายรวม,ค่าบริการ
0,Cafe@Home สี่ยอดกุมารเฮง,50,7.50
1,Cocobee Silpakorn,326,48.90
2,Tiger Pizza,268,40.20
3,Homemade and Tasty (H&T),638,95.70
4,Cottage Coffee & Cake by Vava,367,55.05
5,Baboo bear milk tea -ม.ศิลปากร นครปฐม,245,36.75
6,J&S Restaurant,178,26.70
7,โจ๊กบางกอก สาขาอำเภอเมืองนครปฐม,120,18.00
8,จั๊บจับใจ นครปฐม,550,82.50
9,ข้าวต้ม สะแกทอง,105,15.75


In [13]:
selected_list = [
    'Chinta Cafe'
]

In [17]:
float_shop_list = []
shop_list = []
selected_shop_list = []


float_shop_list = sorted(set(shop_df[ shop_df['ชื่อร้านค้า'].isin(float_list) ]['ชื่อร้านค้า'].to_list()))
shop_list = sorted(set(shop_df[ ~shop_df['ชื่อร้านค้า'].isin(float_list) ]['ชื่อร้านค้า'].to_list()))
# selected_shop_list = sorted(set(shop_df[ shop_df['ชื่อร้านค้า'].isin(selected_list) ]['ชื่อร้านค้า'].to_list()))

In [19]:
float_shop_list

[]

In [20]:
shop_list

['Baboo bear milk tea -ม.ศิลปากร นครปฐม',
 'Cafe@Home สี่ยอดกุมารเฮง',
 'Cocobee Silpakorn',
 'Cottage Coffee & Cake by Vava',
 'Homemade and Tasty (H&T)',
 'J&S Restaurant',
 'KING Sushi&Sashimi',
 'OHO SUSHI สาขานครปฐม',
 'Tiger Pizza',
 'Tokyo Cafe Kitchen (โตเกียว คาเฟ่ คิทเช่น)',
 'กาญจนาไก่ย่างส้มตำ ปลาเผา',
 'ข้าวต้ม ต้นสน อาอี่',
 'ข้าวต้ม สะแกทอง',
 'ข้าวหมกไก่ ข้าวมันไก่ สะพานซอย 2',
 'จั๊บจับใจ นครปฐม',
 'ซูชิจังพ่นไฟ',
 'เวียดนามเฮ้าส์',
 'โจ๊กบางกอก สาขาอำเภอเมืองนครปฐม']

In [21]:
selected_shop_list

[]

In [24]:
from tqdm import tqdm

for i, shop in enumerate( tqdm(shop_list)):
    process(shop, start_invoice_number)
    start_invoice_number+=1

for i, shop in enumerate( tqdm(float_shop_list)):
    process_float(shop, start_invoice_number)
    start_invoice_number+=1

for i, shop in enumerate( tqdm(selected_shop_list)):
    process_float(shop, start_invoice_number)
    start_invoice_number+=1

100%|██████████| 18/18 [00:01<00:00, 12.68it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [25]:
print(start_invoice_number)
with open('number.txt', 'w') as f:
    f.write(f'{start_invoice_number}')

1041


In [26]:
pdf_list = sorted( os.path.join(PDF_DIR, file) for file in os.listdir(PDF_DIR) if file.endswith('.pdf')  and not file.startswith('.'))
len(pdf_list)

18

In [28]:
from pdf2image import convert_from_path

In [29]:
for pdf_path in tqdm(pdf_list):
    name = pdf_path.split(os.sep)[-1][:-4]
    image_path = [ os.path.join(IMAGES_DIR, f'{name}_{i}.png') for i in range(1,3)]
    images = convert_from_path(pdf_path)
    for image, path in zip(images, image_path):
        image.save(path)

100%|██████████| 18/18 [00:12<00:00,  1.47it/s]
